In [49]:
import copy
 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm

In [50]:
# Read Data
spectrum_train = pd.read_excel('spectrum_train.xlsx')
spectrum_test = pd.read_excel('spectrum_valid.xlsx')
temp_train = pd.read_excel('temp_train.xlsx')
temp_test = pd.read_excel('temp_valid.xlsx')

In [51]:
# Convert to np array
X_train = spectrum_train.to_numpy()
y_train = temp_train.to_numpy()
X_test = spectrum_test.to_numpy()
y_test = temp_test.to_numpy()

In [52]:
# convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float64)
y_train_tensor = torch.tensor(X_test, dtype=torch.float64).reshape(-1, 1)
X_test_tensor = torch.tensor(X_train, dtype=torch.float64)
y_test_tensor = torch.tensor(y_test, dtype=torch.float64).reshape(-1, 1)

In [53]:
# define model
model = nn.Sequential(
    nn.Linear(66, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 11)
)

In [54]:
# loss function and optimizer
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

n_epochs = 100
batch_size = 10
batch_start = torch.arange(0, len(X_train_tensor), batch_size)

In [55]:
# hold the best model
best_mse = np.inf
best_weights = None
history = []

In [56]:
# train model
for epoch in range(n_epochs):
    model.train()
    with tqdm.tqdm(batch_start, unit='batch', mininterval=0, disable=True) as bar:
        bar.set_description(f"Epoch {epoch}")
        for start in bar:
            # take a batch
            X_batch = X_train_tensor[start:start + batch_size]
            y_batch = y_train_tensor[start:start + batch_size]
            # forward pass
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # print progress
            bar.set_postfix(mse=float(loss))
    # evaluate accuracy at end of each epoch
    model.eval()
    y_pred = model(X_test_tensor)
    mse = loss_fn(y_pred, y_test_tensor)
    mse = float(mse)
    history.append(mse)
    if mse < best_mse:
        best_mse = mse
        best_weights = copy.deepcopy(model.state_dict())

RuntimeError: mat1 and mat2 must have the same dtype